In [1]:
import datetime as dt
import pandas as pd

from de.processor.source.dart import DART_FUNDAMENTAL_PROCESSOR
from de.processor.source.fdr import FDR_INFO_PROCESSOR, FDR_OHLCV_PREPROCESSOR

from de.processor.factor.processor import (
    FUNDAMENTAL_FACTOR_PROCESSOR,
    MOVING_AVERAGE_FACTOR_PROCESSOR,
    LINEAR_COEF_FACTOR_PROCESSOR,
)

from de.processor.factor.analyser import FACTOR_ANALYSER
from de.model_parameter import MODEL_PARAMTER_PROCESSOR

In [2]:
CFG = {
    "FUNDAMENTAL": {
        "PROCESSOR": {
            "ohlcv_recent_n": 10,
        },
    },
    "LINEAR_COEF": {
        "PROCESSOR": {
            "factors": ["Close", "VolumeRotation"],
            "moving_average_window": 10,
            "coef_recent_n": 20,
        }
    },
    "MOVING_AVERAGE": {
        "PROCESSOR": {
            "factors": ["Close", "VolumeRotation"],
            "short_term_window": round((7 * 2) * (5 / 7)),
            "long_term_window": round((7 * 8) * (5 / 7)),
            "signal_recent_n": 20,
        }
    },
    "FACTOR_ANALYSIS": {
        "profit_recent_n": 20,
    },
}
recen_n = CFG["FACTOR_ANALYSIS"]["profit_recent_n"]
analysis_date = (dt.date.today() - dt.timedelta(days=recen_n)).strftime("%Y-%m-%d")
CFG["analysis_date"] = analysis_date

In [3]:
# Loader

In [4]:
## dart_fundamental
dart_fundamental_df = pd.read_csv("./de/data/dart_fundamental_df.csv", index_col=0)
dart_fundamental_processor = DART_FUNDAMENTAL_PROCESSOR(dart_fundamental_df)

In [5]:
## fdr_info
fdr_info_df = pd.read_csv("./de/data/fdr_info_df.csv", index_col=0)
fdr_info_processor = FDR_INFO_PROCESSOR(fdr_info_df)

In [6]:
## fdr_ohlcv
fdr_ohlcv_df = pd.read_csv("./de/data/fdr_ohlcv_df.csv", index_col=0)
fdr_ohlcv_preprocessor = FDR_OHLCV_PREPROCESSOR(fdr_ohlcv_df)
fdr_ohlcv_df = fdr_ohlcv_preprocessor.get_pps_fdr_ohlcv_df()

# split ohlcv
future_ohlcv_df = fdr_ohlcv_df[CFG["analysis_date"] < fdr_ohlcv_df["Date"]].copy()
past_ohlcv_df = fdr_ohlcv_df[fdr_ohlcv_df["Date"] < CFG["analysis_date"]].copy()

In [7]:
# Parameter
model_paramter_processor = MODEL_PARAMTER_PROCESSOR(CFG["FACTOR_ANALYSIS"])

In [8]:
# Fundamental

# fundamental_factors_df
fundamental_factor_processor = FUNDAMENTAL_FACTOR_PROCESSOR(
    past_ohlcv_df, dart_fundamental_processor, fdr_info_processor
)
fundamental_factors_df = fundamental_factor_processor.get_factor_df(CFG["FUNDAMENTAL"]["PROCESSOR"])

# analyser
fundamental_factor_analyser = FACTOR_ANALYSER(fundamental_factors_df)

fundamental_params_dict = model_paramter_processor.get_params_dict(future_ohlcv_df, fundamental_factor_analyser)
fundamental_params_df = pd.DataFrame().from_dict(fundamental_params_dict, orient="index", columns=["Value", "Variance"])
fundamental_params_df.nlargest(10, "Value").nsmallest(5, "Variance")

,Value,Variance
"((TEPP, 0.5, 0.8), (NPP, 0.3, 0.6))",0.076728,0.002012
"((TAPP, 0.7, 1.0), (CLR, 0.4, 0.7))",0.077017,0.002106
"((CAPP, 0.5, 0.8), (CLR, 0.2, 0.5))",0.077718,0.004195
"((TLR, 0.6, 0.9), (CLR, 0.0, 0.3))",0.091851,0.015207
"((TLR, 0.5, 0.8), (CLR, 0.0, 0.3))",0.098374,0.015595


In [9]:
# Technical

# linear_coef_factors_df
linear_coef_factor_processor = LINEAR_COEF_FACTOR_PROCESSOR(past_ohlcv_df, fdr_info_processor)
linear_coef_factors_df = linear_coef_factor_processor.get_factor_df(CFG["LINEAR_COEF"]["PROCESSOR"])

# Analysis
linear_coef_factor_analyser = FACTOR_ANALYSER(linear_coef_factors_df)

linear_coef_params_dict = model_paramter_processor.get_params_dict(future_ohlcv_df, linear_coef_factor_analyser)
linear_coef_params_df = pd.DataFrame().from_dict(linear_coef_params_dict, orient="index", columns=["Value", "Variance"])
linear_coef_params_df.nlargest(10, "Value").nsmallest(5, "Variance")

,Value,Variance
"((VolumeRotation, 0.6, 0.9), (Close, 0.0, 0.3))",0.082217,0.001013
"((VolumeRotation, 0.4, 0.7), (Close, 0.0, 0.3))",0.082993,0.001052
"((VolumeRotation, 0.6, 0.9), (Close, 0.1, 0.4))",0.086338,0.001104
"((VolumeRotation, 0.5, 0.8), (Close, 0.0, 0.3))",0.086835,0.001258
"((VolumeRotation, 0.7, 1.0), (Close, 0.0, 0.3))",0.088703,0.004183


In [10]:
# Technical

# moving_average_factors_df
moving_average_factor_processor = MOVING_AVERAGE_FACTOR_PROCESSOR(past_ohlcv_df, fdr_info_processor)
moving_average_factors_df = moving_average_factor_processor.get_factor_df(
    CFG["MOVING_AVERAGE"]["PROCESSOR"]
)

# Analysis
moving_average_factor_analyser = FACTOR_ANALYSER(moving_average_factors_df)

moving_average_params_dict = model_paramter_processor.get_params_dict(future_ohlcv_df, moving_average_factor_analyser)
moving_average_params_df = pd.DataFrame().from_dict(moving_average_params_dict, orient="index", columns=["Value", "Variance"])
moving_average_params_df.nlargest(10, "Value").nsmallest(5, "Variance")

,Value,Variance
"((VolumeRotation, 0.0, 0.3), (Close, 0.0, 0.3))",0.074011,0.000965
"((VolumeRotation, 0.3, 0.6), (Close, 0.1, 0.4))",0.077013,0.000971
"((VolumeRotation, 0.4, 0.7), (Close, 0.0, 0.3))",0.078751,0.000977
"((VolumeRotation, 0.1, 0.4), (Close, 0.1, 0.4))",0.078810,0.000989
"((VolumeRotation, 0.1, 0.4), (Close, 0.0, 0.3))",0.080033,0.001074
